In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyyaml pandas scikit-learn albumentations segmentation-models-pytorch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.7 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from scipy import ndimage

# --- ⚙️ Configuration ---
CSV_PATH = '/content/drive/MyDrive/CAF-GAN/data/splits/master_subset_2k.csv' # Or your full dataset CSV
IMAGE_DIR = '/content/drive/MyDrive/CAF-GAN/mimic-cxr-jpg-2.0.0/files/'
MASK_DIR = '/content/drive/MyDrive/CAF-GAN/data/masks_512x512/' # New directory for high-res masks
MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/CAF-GAN/lung_segmentation_data/outputs/best_lung_segmenter.pth' # <--- PATH TO YOUR TRAINED MODEL WEIGHTS
IMG_SIZE = 512

def preprocess_image(image_path, transform):
    """Loads and preprocesses a single image for inference."""
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)

    if transform:
        augmented = transform(image=image_np)
        image_tensor = augmented['image']

    return image_tensor.unsqueeze(0)

def main():
    """Generates high-resolution segmentation masks using a pre-trained model."""
    print("🚀 Starting high-resolution mask generation...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # --- Model Definition ---
    # This model should be identical to the one you trained for lung segmentation.
    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights=None,  # Weights will be loaded from your trained file
        in_channels=3,
        classes=1,
    )

    # --- Load Your Trained Weights ---
    # This is the most important step!
    print(f"💾 Loading trained weights from: {MODEL_WEIGHTS_PATH}")
    # Ensure the model is loaded correctly for the device
    model.load_state_dict(torch.load(MODEL_WEIGHTS_PATH, map_location=device))
    model.to(device)
    model.eval()

    # --- Transformation Pipeline for Inference ---
    # This should only include resizing and normalization, not augmentations like flips/rotations.
    transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE, interpolation=Image.LANCZOS),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # --- Data Handling ---
    df = pd.read_csv(CSV_PATH)
    os.makedirs(MASK_DIR, exist_ok=True)
    print(f"🖼️ Found {len(df)} images to process.")
    print(f"💾 Masks will be saved to {MASK_DIR} at {IMG_SIZE}x{IMG_SIZE} resolution.")

    successful_generations = 0
    failed_generations = 0

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating Masks"):
        subject_id = str(row['subject_id'])
        study_id = str(row['study_id'])
        dicom_id = row['dicom_id']

        jpg_path = os.path.join(
            IMAGE_DIR, f'p{subject_id[:2]}', f'p{subject_id}', f's{study_id}', f'{dicom_id}.jpg'
        )

        if not os.path.exists(jpg_path):
            failed_generations += 1
            continue

        try:
            image_tensor = preprocess_image(jpg_path, transform).to(device)

            with torch.no_grad():
                predicted_mask = model(image_tensor)

            # --- Post-processing for a clean mask ---
            probabilities = torch.sigmoid(predicted_mask).squeeze().cpu().numpy()
            binary_mask = (probabilities > 0.5).astype(np.uint8)

            # Morphological operations to remove small artifacts and fill holes
            binary_mask = ndimage.binary_closing(binary_mask, structure=np.ones((5,5))).astype(np.uint8)
            binary_mask = ndimage.binary_opening(binary_mask, structure=np.ones((3,3))).astype(np.uint8)

            # Convert back to an image format for saving
            mask_to_save = (binary_mask * 255)
            mask_image = Image.fromarray(mask_to_save)

            output_path = os.path.join(MASK_DIR, f"{dicom_id}.png")
            mask_image.save(output_path)
            successful_generations += 1

        except Exception as e:
            print(f"❌ Error processing {jpg_path}: {e}")
            failed_generations += 1

    print("\n✅ Mask generation complete!")
    print(f"📈 Success: {successful_generations}, Failed: {failed_generations}")

if __name__ == "__main__":
    main()

🚀 Starting high-resolution mask generation...
Using device: cuda
💾 Loading trained weights from: /content/drive/MyDrive/CAF-GAN/lung_segmentation_data/outputs/best_lung_segmenter.pth
🖼️ Found 2000 images to process.
💾 Masks will be saved to /content/drive/MyDrive/CAF-GAN/data/masks_512x512/ at 512x512 resolution.


Generating Masks: 100%|██████████| 2000/2000 [27:54<00:00,  1.19it/s]


✅ Mask generation complete!
📈 Success: 2000, Failed: 0
